In [ ]:
import pandas as pd

# Import all blocks as pandas DataFrames
blocks = []
for i in range(1, 25):
    block = pd.read_csv(f"block{i}.csv")
    blocks.append(block)

# Exclude saccades according to the following criteria:
# automatically rejected by LatencyMeter, latency <100ms or >1000ms, 
# amplitude >40◦, peak velocity > 1000 deg s−1
for block in blocks:
    block = block[(block["LatencyMeter"] != "rejected") & 
                  (block["latency"] >= 100) & (block["latency"] <= 1000) &
                  (block["amplitude"] <= 40) & (block["peak_velocity"] <= 1000)]

# For each participant, gather data from all blocks, 
# numbering trials sequentially according to the protocol, i.e. in the range 1–240
all_data = pd.concat(blocks)
all_data["trial_number"] = range(1, len(all_data) + 1)

# For raw data, extract the first 25 samples for each saccade 
# (i.e. −25ms to 100ms from saccade initiation)
all_data["saccade_data"] = all_data.apply(lambda row: row["raw_data"][:25], axis=1)

# where <25 samples are available, right-pad the data with
# the amplitude of the last observed sample
all_data["saccade_data"] = all_data.apply(lambda row: 
                                        row["saccade_data"] + [row["saccade_data"][-1]]*(25-len(row["saccade_data"])), axis=1)

# Divide by the target eccentricity (10◦ was used throughout) to obtain 25 gain samples
all_data["gain_samples"] = all_data.apply(lambda row: [sample/10 for sample in row["saccade_data"]], axis=1)
